# Домашнее задание 6 (демо).<br> Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина
<img src='../../mlcourse.ai/img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [2]:
data = pd.read_csv('../../mlcourse.ai/data/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [5]:
y = data['quality']

X_train, X_holdout, y_train, y_holdout = train_test_split(data.drop(['quality'], axis=1), y, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [6]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
y_pred = linreg.predict(X_holdout_scaled)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [8]:
print("Mean squared error (train) {:.3f}".format(mean_squared_error(y_train, linreg.predict(X_train_scaled))))
print("Mean squared error (test): {:.3f}".format(mean_squared_error(y_holdout, y_pred)))

Mean squared error (train) 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [9]:
linreg_coef = pd.DataFrame(abs(linreg.coef_)).T # берем коэф. по модулю
linreg_coef.columns=X_train.columns
linreg_coef.T.sort_values(by=[0])

,0
citric acid,0.000183
chlorides,0.008127
total sulfur dioxide,0.014304
free sulfur dioxide,0.042180
sulphates,0.062053
fixed acidity,0.097822
alcohol,0.129533
pH,0.150036
volatile acidity,0.192260
residual sugar,0.538164


параметр __density__ больше остальных влияет на предсказание

## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [10]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=17,
      selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [11]:
lasso1_coef = pd.DataFrame(abs(lasso1.coef_)).T # берем по модулю
lasso1_coef.columns = X_train.columns
lasso1_coef.T.sort_values(by=[0]).T

,fixed acidity,citric acid,total sulfur dioxide,chlorides,sulphates,free sulfur dioxide,pH,volatile acidity,density,residual sugar,alcohol
0,0.0,0.0,0.0,0.002747,0.029722,0.043088,0.067277,0.188479,0.235492,0.256363,0.322425


In [12]:
print("признак '{}' отвалится первым".format(lasso1_coef.columns[0]))

признак 'fixed acidity' отвалится первым


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [13]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
        verbose=False)

In [14]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [15]:
lasso_cv_coef = pd.DataFrame(abs(lasso_cv.coef_)).T
lasso_cv_coef.columns = X_train.columns
lasso_cv_coef= lasso_cv_coef.T.sort_values(by=[0]).T
lasso_cv_coef

,citric acid,chlorides,total sulfur dioxide,free sulfur dioxide,sulphates,fixed acidity,alcohol,pH,volatile acidity,residual sugar,density
0,0.0,0.006933,0.012969,0.042698,0.060939,0.093295,0.137115,0.146549,0.192049,0.526883,0.648161


In [16]:
print('Признак "{}" в модели Lasso обнулился первым'.format(lasso_cv_coef.columns[0]))

Признак "citric acid" в модели Lasso обнулился первым


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [17]:
print("Mean squared error (train) {:.3f}".format(mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))))
print("Mean squared error (test) {:.3f}".format(mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))))

Mean squared error (train) 0.558
Mean squared error (test) 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [18]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [23]:
print("Mean squared error (train) {:.2f}".format(mean_squared_error(y_train, forest.predict(X_train_scaled))))
print(
    "Mean squared error (cv): {:.2f}".format( abs(cross_val_score(
        forest, X_holdout_scaled, y_holdout, scoring='neg_mean_squared_error'
    ).mean() )))
print("Mean squared error (test) {:.2f}".format(mean_squared_error(y_holdout, forest.predict(X_holdout_scaled))))

Mean squared error (train) 0.05
Mean squared error (cv): 0.45
Mean squared error (test) 0.37


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [25]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=17),
                                   forest_params, 
                                   scoring='neg_mean_squared_error',  
                                   n_jobs=-1, 
                                   verbose=True)

locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 3150 out of 3150 | elapsed: 21.6min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_job

In [27]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 -0.39773288191505934)

In [28]:
mean_squared_error(y_holdout, locally_best_forest.predict(X_holdout_scaled))

0.36572455603132475

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [29]:
model_rf = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
model_rf.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=19, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

In [30]:
cv_train = cross_val_score(model_rf, X_train_scaled, y_train, scoring='neg_mean_squared_error')
cv_holdout = cross_val_score(model_rf, X_holdout_scaled, y_holdout, scoring='neg_mean_squared_error')

In [34]:
print("Mean squared error (cv) {:.2f}".format(abs(cv_holdout.mean())))
print("Mean squared error (test) {:.2f}".format(mean_squared_error(y_holdout, model_rf.predict(X_holdout_scaled))))

Mean squared error (cv) 0.45
Mean squared error (test) 0.37


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [35]:
rf_importance = pd.DataFrame(abs(model_rf.feature_importances_)).T # берем по модулю
rf_importance.columns = X_train.columns
rf_importance = rf_importance.T.sort_values(by=[0]).T
rf_importance

,sulphates,citric acid,fixed acidity,chlorides,total sulfur dioxide,residual sugar,pH,density,free sulfur dioxide,volatile acidity,alcohol
0,0.059499,0.0615,0.061873,0.067819,0.071644,0.072529,0.072646,0.08653,0.113509,0.117627,0.214825


In [36]:
rf_importance.columns[-1]

'alcohol'

**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

Итоговые значения ошибок моделей на отложенной выборке:

In [43]:
print("Linear Regression : {:.2f}".format(mean_squared_error(
    y_holdout, linreg.predict(X_holdout_scaled))))
print("Lasso Regression {:.2f}".format(
    mean_squared_error(
        y_holdout, lasso_cv.predict(X_holdout_scaled))))
print("Random Forest {:.2f}".format(
    mean_squared_error(
        y_holdout, model_rf.predict(X_holdout_scaled))))

Linear Regression : 0.58
Lasso Regression 0.58
Random Forest 0.37


Судя по всему, зависимость качества вина от имеющихся признаков не линейная, поэтому случайный лес работает тут лучше, чем линейные модели